Classifier

In [1]:
import pandas as pd
import csv

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [3]:
df = pd.read_csv('Dataset.csv')

In [4]:
df.shape

(991, 4)

In [5]:
X = df['Article_content']
Y = df['type']

In [6]:
X = X.fillna("")

In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
Y = le.fit_transform(Y.astype(str))


In [8]:
from sklearn.model_selection import cross_val_score

In [9]:
from sklearn.pipeline import Pipeline

In [10]:
from sklearn.ensemble import RandomForestClassifier

In [63]:
text_clf_RandomForest = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf',  RandomForestClassifier(n_estimators=200,oob_score=True,random_state=100,n_jobs=-1,max_features=None,min_samples_leaf=50))])

calculate score for each CV split

In [64]:
scores5 = cross_val_score(text_clf_RandomForest, X, Y, cv=10, scoring='accuracy')

In [65]:
scores5.mean()

0.997979797979798

To make predictions

In [66]:
# train the model using all the data
text_clf_RandomForest.fit(X, Y)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...imators=200, n_jobs=-1,
            oob_score=True, random_state=100, verbose=0, warm_start=False))])

In [67]:
len(text_clf_RandomForest.named_steps['vect'].get_feature_names())

19336

Articles extracted from RSS

In [16]:
import feedparser
# RSS feed url
url = 'http://www.dailymirror.lk/RSS_Feeds/breaking-news'
feed = feedparser.parse(url)
entries = feed['entries']

from newspaper import Article
articles = []
for entry in entries:
    if entry:
        url = entry['link']
        article = Article(url)
        article.download()
        article.parse()
        articles.append({
            'Title': entry['title'],
            'Date': entry['published'],
            'Article_content': article.text,
            'Summary': entry['summary'],
            'Link': entry['link'],
        })
import re
TAG_RE=re.compile(r'<[^>]+>')
for article in articles:
    summary = article['Summary']
    pure_summary=TAG_RE.sub('',summary)
    article['Summary']=pure_summary
import csv
with open('articles_from_rss.csv',mode='w') as csv_file:
    fieldnames=['Title','Date','Article_content','Summary','Link']
    writer=csv.DictWriter(csv_file,fieldnames=fieldnames)

    writer.writeheader()
    for article in articles:
        writer.writerow({'Title':article['Title'].encode("utf-8"),'Date':article['Date'].encode("utf-8"),'Article_content':article['Article_content'].encode("utf-8"),'Summary':article['Summary'].encode("utf-8"),'Link':article['Link'].encode("utf-8")})


In [17]:
df_new = pd.read_csv('articles_from_rss.csv', )

In [18]:
new_articles = df_new['Article_content']

In [19]:
df_new.shape

(20, 5)

In [20]:
new_articles = new_articles.fillna("")

Make Predictions

In [21]:
type_arr = text_clf_RandomForest.predict(new_articles)
print(type_arr)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


Add Dengue articles to the database

In [22]:
from mysql.connector import MySQLConnection, Error

In [23]:
config = {
  'user': 'root',
  'password': '',
  'host': '127.0.0.1',
  'database': 'dengue_sri_lanka',
  'raise_on_warnings': True
}

In [24]:
def insert_article(publication_date, title, content, summary, link):
    query = "INSERT IGNORE INTO news_articles(publication_date, title, content, summary, link) " \
            "VALUES(%s,%s,%s,%s,%s)"
    args = (publication_date, title, content, summary, link)
 
    try:
        conn = MySQLConnection(**config)
 
        cursor = conn.cursor()
        cursor.execute(query, args) 
        conn.commit()
    except Error as error:
        print(error)
 
    finally:
        cursor.close()
        conn.close()

In [25]:
# extract articles with 0 as the result and update the db table (without duplicates)
index = 0;
for x in type_arr:
    if x==0: 
        insert_article(articles[index]['Date'], articles[index]['Title'], articles[index]['Article_content'], articles[index]['Summary'], articles[index]['Link'])

    index = index + 1

In [26]:
# the code for info extraction (use above Dengue articles and extract data from that and store in respective tables)

In [27]:
import re

In [28]:
from word2number import w2n

In [29]:
regex_for_publication_date = re.compile(r"""(?:\d{4}-\d{2}-\d{2})|(?:\d{1,2}\s*(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|April|May|June|July|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s*\d{4})""",re.IGNORECASE)

In [30]:
regex_for_cases = re.compile(r'(\d+(?:(?:,|\s|(?:,\s)|(?:\s,))\d{1,3}){0,1})\s*(?:dengue\scases|dengue\sfever\scases|cases|suspected\sdengue\scases|suspected\scases|suspected\sdengue\sfever\scases|patients|dengue\spatients|from)', re.IGNORECASE)

In [31]:
regex_for_deaths = re.compile(r"""((?:\d+)|(?:[^\S]\d{1,3}(?:(?:,|\s|(?:,\s)|(?:\s,))\d+)+)|
(?:one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|
(?:(?:(?:twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety)\s*)(?:one|two|three|four|five|six|seven|eight|nine)?)))
\s*
(?:deaths|death|dead\b|(?:people\s*|patients\s*)?(?:have\s*|had\s*)?(?:died|dead\b)|dengue(?:\s|-)related\sdeath)
""", re.VERBOSE | re.IGNORECASE)

In [32]:
# regex to extract sentences having no. of cases/deaths
regex_for_sentences = re.compile(r"[^.]*?(?:\d+(?:(?:,|\s|(?:,\s)|(?:\s,))\d+)*\s*(?:dengue\scases|dengue\sfever\scases|cases|suspected\sdengue\scases|suspected\scases|suspected\sdengue\sfever\scases)|(?:(?:(?:\d+(?:(?:,|\s|(?:,\s)|(?:\s,))\d+)*)|(?:one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|(?:(?:(?:twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety)\s*)(?:one|two|three|four|five|six|seven|eight|nine)?)))\s*(?:deaths|death|dead\b|(?:people\s*|patients\s*)?(?:have\s*|had\s*)?(?:died|dead\b)|patients|dengue\srelated\sdeath)))[^.]*\.")

In [33]:
# regex_for_years 20xx
regex_for_years = re.compile(r"""\b20(?:\d{2}){1}\b|this\syear""", re.IGNORECASE)

In [34]:
# regex for months
regex_for_months = re.compile(r'Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|April|May|June|July|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)')

In [35]:
# regex for "so far"
regex_for_so_far = re.compile(r'so\sfar')

In [36]:
# regex to catch all time periods
regex_for_time_period_of = re.compile(r"""((?:\b20(?:\d{2}){1}\b|this\syear)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|April|May|June|July|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember))|(?:so\sfar))""", re.IGNORECASE)

In [37]:
# regex for districts
regex_for_districts = re.compile(r'Jaffna|Kilinochchi|Mannar|Mullaitivu|Vavuniya|Puttalam|Kurunegala|Gampaha|Colombo|Kalutara|Anuradhapura|Polonnaruwa|Matale|Kandy|Nuwara\sEliya|Kegalle|Ratnapura|Trincomalee|Trinco|Batticaloa|Ampara|Badulla|Monaragala|Hambantota|Matara|Galle', re.IGNORECASE)

In [38]:
# regex for provinces
regex_for_provinces = re.compile(r'Western|Central|Eastern|North\sCentral|Northern|North\sWestern|Sabaragamuwa|Southern|Uva')

In [39]:
# regex for countries
regex_for_country = re.compile(r'Sri Lanka|island|country|countrywide')

In [40]:
# regex for different places
regex_for_place_of = re.compile(r'((?:Jaffna|Kilinochchi|Mannar|Mullaitivu|Vavuniya|Puttalam|Kurunegala|Gampaha|Colombo|Kalutara|Anuradhapura|Polonnaruwa|Matale|Kandy|Nuwara\sEliya|Kegalle|Ratnapura|Trincomalee|Trinco|Batticaloa|Ampara|Badulla|Monaragala|Hambantota|Matara|Galle)|(?:Western|Central|Eastern|North\sCentral|Northern|North\sWestern|Sabaragamuwa|Southern|Uva)|(?:Sri Lanka|island|country|countrywide))', re.IGNORECASE)

In [41]:
def get_sentences(text):
    return regex_for_sentences.findall(text)

In [42]:
regex_for_dates = re.compile(r"""(?:\d{1,2}\s*(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|April|May|June|July|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s*\d{4})""",re.IGNORECASE)

In [43]:
def get_month_in_number(month):
    m = {
        'jan': 1,
        'feb': 2,
        'mar': 3,
        'apr':4,
        'may':5,
        'jun':6,
        'jul':7,
        'aug':8,
        'sep':9,
        'oct':10,
        'nov':11,
        'dec':12
        }
    s = month.strip()[:3].lower()

    try:
        out = m[s]
        return out
    except:
        raise ValueError('Not a month')
    

In [44]:
def get_published_date(text):
    dates = regex_for_publication_date.findall(text)
    for index,date in enumerate(dates):
        if(regex_for_dates.match(date)):
            day = re.search('\d{1,2}', date).group()
            month = get_month_in_number(re.search('\D{3,9}', date).group())
            year = re.search('\d{4}', date).group()
            date = year + "-" + str(month) + "-" + day
            dates[index] = date
    
    return dates

In [45]:
def get_cases(text):
    cases = regex_for_cases.findall(text)
    for index,case in enumerate(cases):
        case = case.replace(" ","")
        case = case.replace(",","")
        case = int(case)   
        cases[index] = case
    return cases

In [46]:
regex_for_digits = re.compile(r'(?:(?:\s)*\d+(?:(?:\s)\d+)*)')

In [47]:
def get_deaths(text):
    deaths = regex_for_deaths.findall(text)
    for index,death in enumerate(deaths):
        if(regex_for_digits.match(death)):
            death = death.replace(" ","")
            death = death.replace(",","")
        death = w2n.word_to_num(death)   
        
        deaths[index] = death
    return deaths        

In [48]:
def get_years(text,text_date):
    years = regex_for_years.findall(text)    
    for index,year in enumerate(years):
        if(year.lower() == "this year"):
            pub_date = get_published_date(text_date)[0]
            year = re.search('\d{4}',pub_date).group()
            years[index] = year
    return years
            

In [49]:
def get_months(text):
    return regex_for_months.findall(text)

In [50]:
def get_so_far(text):
    return regex_for_so_far.findall(text)

In [51]:
def get_time_period_of(text):
    return regex_for_time_period_of.findall(text)

In [52]:
def get_provinces(text):
    return regex_for_provinces.findall(text)

In [53]:
def get_districts(text):
    return regex_for_districts.findall(text)

In [54]:
def get_country(text):
    return regex_for_country.findall(text)

In [55]:
def get_place_of(text):
    return regex_for_place_of.findall(text)

In [56]:
# row = [published_date, cases, deaths, year, month, time, province, countrywide, district]
def get_info(published_date, cases, deaths, time_periods, places):
    lst = []
    for index,case in enumerate(cases):
        row = []
        if(len(published_date) != 0):    
            row.append(published_date[0])
        else:
            row.append("")
        row.append(case)
        if(len(deaths) > index):
            row.append(deaths[index])
        else:
            row.append(0)
#         time_period
        if(len(time_periods) > index):
            if(len(get_years(time_periods[index], published_date[0])) != 0):
                row.append(get_years(time_periods[index], published_date[0])[0])
                row.append("")
                row.append("")
            elif(len(get_months(time_periods[index])) != 0):
                row.append("")
                row.append(get_months(time_periods[index])[0])
                row.append("")
            else:
                row.append("")
                row.append("")
                row.append(time_periods[index])
           
        else:
            row.append("")   
            row.append("")   
            row.append("")   

#        places
        if(len(places) > index):
        
            if(len(get_provinces(places[index])) != 0):
                row.append(get_provinces(places[index])[0])   
                row.append("")   
                row.append("") 
            elif(len(get_country(places[index])) != 0):
                row.append("")   
                row.append(get_country(places[index])[0])   
                row.append("")
            elif(len(get_districts(places[index])) != 0):
                row.append("")   
                row.append("")   
                row.append(get_districts(places[index])[0]) 
            else:
                row.append("")   
                row.append("")   
                row.append("")  
        
#             row.append(places[index])
        else:
            row.append("")   
            row.append("")   
            row.append("")  
        lst.append(row)
    return lst
    
    

In [58]:
# Insert extracted data into table
conn = MySQLConnection(**config)
cursor = conn.cursor()


for index,x in enumerate(type_arr):
    if x==0: #if article is Dengue
        row = articles[index]

        
        
        for sentence in get_sentences(row['Article_content']):
            lst = get_info(get_published_date(row['Date']),get_cases(sentence),get_deaths(sentence),get_time_period_of(sentence),get_place_of(sentence))
            for i,l1 in enumerate(lst):
#                 print(l1)

                query = "INSERT INTO regex_output(publication_date, cases, deaths, year, month, time_duration, province, countrywide, article_id, district) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

                args = (l1[0], l1[1], l1[2], l1[3], l1[4], l1[5], l1[6], l1[7], 7, l1[8])

                try:
                    cursor.execute(query, args)
                    conn.commit()

                except Error as error:
                    print(error)
                
cursor.close()
conn.close()
    